# Exploration & data typing.
This jupyter notebook will be used to explore the given data and come up with sensible approaches to pre-process the data. After a process has been established, the actual analysis will be implemented in separate python files.


## Exploration
We're using this Notebook to get ourselves familiar with the data set. From that we will make decisions to best pre-process the data and/or what we could vary for potential (hyper) parameter tuning to get the most accurate model.
1. First read in the data and explore it:
    * Confirm things are in the right datatype (dates are dates, ints are ints, etc.)
    * Confirm the categorical values have only the values as specified.
    * Check which columns have empty or N/A values and figure out how to address them.
    * See if and how to convert strings/objects to numerical data types.
2. Explore ways above possible problems have occurred and fix them.
3. After fixing the data, visualize all the data to get a gross understanding of what affects the price.

In [20]:
import pandas as pd
from pathlib2 import Path

# Assuming dataset is in ~/data, using pathlib to make compatible over different OS
data_file = Path("data") / "Stryd_Dataset.csv"
df_houses = pd.read_csv(data_file)

# Get more information and description of the data to spot potential outliers, or other odd things.
df_houses.info()
print(df_houses.describe())
print(df_houses.describe(include=object))

# Double check to make sure the id column is unique and can be used as index
print(f"The id column is unique: {df_houses['id'].is_unique}")

# Check all the categories:
print(f"floorPosition (Top, High, Middle, Low, Bottom): {df_houses['floorPosition'].unique()}")
print(f"RenovationCondition (4, 3, 2, 1): {df_houses['renovationCondition'].unique()}")
print(f"buildingStructure (6, 5, 4, 3, 2, 1): {df_houses['buildingStructure'].unique()}")
print(f"elevator (1, 0): {df_houses['elevator'].unique()}")
print(f"fiveYearsProperty (1, 0): {df_houses['elevator'].unique()}")
print(f"subway (1, 0): {df_houses['subway'].unique()}")
print(f"district (?): {df_houses['district'].unique()}")

/tmp/ipykernel_9620/3228157002.py:6: DtypeWarning: Columns (0,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_houses = pd.read_csv(data_file)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318851 entries, 0 to 318850
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   318851 non-null  object 
 1   Lng                  318851 non-null  float64
 2   Lat                  318851 non-null  float64
 3   Cid                  318851 non-null  float64
 4   tradeTime            318851 non-null  object 
 5   DOM                  160874 non-null  float64
 6   followers            318851 non-null  int64  
 7   price                318851 non-null  int64  
 8   square               318851 non-null  float64
 9   livingRoom           318851 non-null  object 
 10  drawingRoom          318851 non-null  object 
 11  kitchen              318851 non-null  int64  
 12  bathRoom             318849 non-null  float64
 13  floorPosition        317580 non-null  object 
 14  floorsCount          318819 non-null  float64
 15  constructionTime 

## Initial overview of the data
Just from the information we can see a few things:
1. There are 318851 house transactions in this dataset.
2. id column has letters, so is object. But is unique so we use as index_col
3. Some columns have missing values:
    * 5, `DOM`, almost half
    * 12, `bahtRoom`, just 2 missing
    * 13, `floorPosition`
    * 14, `floorsCount` less than `floorPosition` too, but probably a lot of overlap --> maybe no floors some?
    * 15, `constructionTime`
    * 19, `elevator`
    * 20, `fiveYearsProperty`
    * 21, `subway`
    * 23, `communityAverage`
4. Some date types are not properly detected automatically or are not what I expected.
    * `TradeTime` needs to be a date object
    * Many ints seem to be mistyped as floats or objects --> gives strong indication we have NaN's and/or other wrong values
5. One bad/inaccurate value already in 'bathRoom', 2011 seems to many for any house.
6. All the categories have more values than described.

`LivingRoom`, `drawingRoom` and `floorPosition` are objects, but should be ints (or even floats would be fine). Let's explore those further:


In [21]:
# Figure out why livingRoom, drawingRoom and floorPosition are objects.
print(df_houses['livingRoom'].unique())
print(df_houses['drawingRoom'].unique())
print(df_houses['floorPosition'].unique())

[2 3 1 4 5 6 9 0 7 8 '1' '4' '3' '2' '5' '7' '9' '6' '0' '#NAME?' '8']
[1 2 0 4 3 5 '0' '1' '2' '3' '4' 'middle 14' 'middle 15' 'middle 16'
 'middle 6' 'high 14' 'top 6' 'low 6' 'low 16' 'high 12' 'low 15' '5'
 'high 6' 'bottom 28' 'bottom 11' 'middle 24' 'bottom 20' 'middle 22']
['high' 'middle' 'bottom' 'low' 'top' nan '钢混结构' '混合结构']


## Strings in some numerical columns (`livingRoom`, `drawingRoom`, `floorPosition`)

It's likely something is consistently going wrong with the data in these columns. Especially for livingRoom all the values could be fine except for the _'#NAME?'_ string. Let's see if we can find out what's going on with rows that have _'#NAME?'_ in livingRoom:

In [22]:
df_name_rows = df_houses.loc[df_houses['livingRoom'] == '#NAME?']
df_name_rows.info()
print(df_name_rows.describe())
print(df_name_rows.describe(include=object))
print("livingRoom:", df_name_rows['livingRoom'].unique())
print("drawingRoom:", df_name_rows['drawingRoom'].unique())
print("floorPosition:", df_name_rows['floorPosition'].unique())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 92235 to 245394
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   32 non-null     object 
 1   Lng                  32 non-null     float64
 2   Lat                  32 non-null     float64
 3   Cid                  32 non-null     float64
 4   tradeTime            32 non-null     object 
 5   DOM                  25 non-null     float64
 6   followers            32 non-null     int64  
 7   price                32 non-null     int64  
 8   square               32 non-null     float64
 9   livingRoom           32 non-null     object 
 10  drawingRoom          32 non-null     object 
 11  kitchen              32 non-null     int64  
 12  bathRoom             30 non-null     float64
 13  floorPosition        32 non-null     object 
 14  floorsCount          0 non-null      float64
 15  constructionTime     32 non-null  

## A consistent issue with _#NAME?_
 Looking at the *32* rows with _'#Name?'_ for livingRoom numbers there is something wrong consistently.
- `drawingRoom` a word and number only.
- year build instead of `bathRoom` numbers (hence the 2011 `bathRooms` we saw earlier)
- Chinese text for `floorPosition` that seems to be words for corresponding with `buildingStructure` numbers (_混合结构_, _钢混结构_) being _2_ and _6_ respectively <-- **I checked if it made a different by using different encodings when reading in the file, but no possible encoding improved things.**
- some following columns being empty (`floorsCount`, `elevator`, `fiveYearsProperty` and `subway`)
- It seems like #Name? is being wrongly displayed instead of a few columns for spaces (`livingRoom`, `drawingRoom`, `kitchen` and `bathroom`), and therefore all the values in the row a shifted back 3 columns.
- Not only that: The number in kitchen is limited to (_1_,_2_,_3_ and _4_), suggesting it is actually the `renovationCondition` column; and the `drawingRoom` text and number is likely `floorPosition` with `floorsCount`.

There could be a way to put a lot of the data in the right columns, however we'd still be missing values for the 4 living spaces. In addition to that we aren't completely sure the above observations are correct. Taken with the fact this is only **32** rows out of **318851** rows, it seems sensible to just remove these rows:

In [23]:
import numpy as np

# renaming it, realizing now every row is rather more a transaction for a house
df_transactions = df_houses.loc[df_houses['livingRoom'] != '#NAME?']
df_transactions.info()
print(df_transactions.describe())
print(df_transactions.describe(include=object))
print(f"floorPosition (Top, High, Middle, Low, Bottom): {df_transactions['floorPosition'].unique()}")
print(f"RenovationCondition (4, 3, 2, 1): {np.sort(df_transactions['renovationCondition'].unique())}")
print(f"buildingStructure (6, 5, 4, 3, 2, 1): {np.sort(df_transactions['buildingStructure'].unique())}")
print(f"elevator (1, 0): {df_transactions['elevator'].unique()}")
print(f"fiveYearsProperty (1, 0): {df_transactions['elevator'].unique()}")
print(f"subway (1, 0): {df_transactions['subway'].unique()}")

print(df_transactions.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318819 entries, 0 to 318850
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   318819 non-null  object 
 1   Lng                  318819 non-null  float64
 2   Lat                  318819 non-null  float64
 3   Cid                  318819 non-null  float64
 4   tradeTime            318819 non-null  object 
 5   DOM                  160849 non-null  float64
 6   followers            318819 non-null  int64  
 7   price                318819 non-null  int64  
 8   square               318819 non-null  float64
 9   livingRoom           318819 non-null  object 
 10  drawingRoom          318819 non-null  object 
 11  kitchen              318819 non-null  int64  
 12  bathRoom             318819 non-null  float64
 13  floorPosition        317548 non-null  object 
 14  floorsCount          318819 non-null  float64
 15  constructionTime 

## Removing 32 rows fixed a lot of the issues
- All the categories are fixed now aside for `floorPosition`. In this category, luckily, we can use the NaN values as a new category meaning 'unknown'.
- The bathRoom max is also more sensible now with 7.

## Addressing the NaN values
The next biggest thing is the issue non-null values we're seeing in the information. We're missing a lot of data in the `DOM` column. Besides that we're missing some data in `floorPosition` (but we decided this is okay), about 10x that in `constructionTime` and a few hundred in `communityAverage`.
There are a few ways to address this, but non is perfect. We can:
1. delete all the rows with missing data. <-- not prefered, will lose half the data because of `DOM`.
2. Delete `DOM` column, and the rows for the other columns missing data. this will lose a lot less data than 1, aside from the column, less than 10% of the data.
3. Substitute all or some of the missing data with 0's (or other value), as we had already decided to do with the `floorPosition` NaN values.
4. Same as 3 but replace with mean or median instead of 0.
5. A combination of 2 and 3 (or 4): Delete `DOM` column, and substitute the NaN values in `ConstructionTime` and `communityAverage` with 0's or means/.
6. Delete all NaN's aside for those in `DOM`. Replace those as described in 3 or 4.
For different models, different approaches could be best.

Let's explore what seems to be the most reasonable place to start:

In [24]:
df_fill_dom_0 = df_transactions['DOM'].fillna(0)
df_fill_dom_mean = df_transactions['DOM'].fillna(29) # is rounded to the closest int
df_fill_dom_median = df_transactions['DOM'].fillna(7)
df_dropna_dom = df_transactions['DOM'].dropna()
for df in (df_dropna_dom, df_fill_dom_0, df_fill_dom_mean, df_fill_dom_median):
    print(df.describe())
    print(df.corr(df_transactions['price']))

count    160849.000000
mean         28.826664
std          50.240049
min           1.000000
25%           1.000000
50%           6.000000
75%          37.000000
max        1677.000000
Name: DOM, dtype: float64
0.21539072171852477
count    318819.000000
mean         14.543487
std          38.485732
min           0.000000
25%           0.000000
50%           1.000000
75%           7.000000
max        1677.000000
Name: DOM, dtype: float64
0.2944749235495125
count    318819.000000
mean         28.912549
std          35.685168
min           1.000000
25%           6.000000
50%          29.000000
75%          29.000000
max        1677.000000
Name: DOM, dtype: float64
0.16941787042651998
count    318819.000000
mean         18.011881
std          37.316420
min           1.000000
25%           6.000000
50%           7.000000
75%           7.000000
max        1677.000000
Name: DOM, dtype: float64
0.26950118495148506


## Revisiting DOM

- There's quite some (0.16 ~ 0.29) correlation between `DOM` and `price`, so it's not a good idea to just delete the column (2 out). Considering the few rows with NaN's in all the other columns. Option 6 seems the most reasonable place to start. Now filling DOM with 0 seems to have the highest correlation with `price` (0.29), so let's go with that.

## Let's clean the data and properly type it

In [25]:
from sklearn.preprocessing import LabelEncoder
# Dictionary with column names and their data types
types = {
    "Cid": "category",
    "floorsCount": "int64",
    "floorPosition": "category",
    "renovationCondition": "category",
    "buildingStructure": "category",
    "livingRoom": "int64",
    "drawingRoom": "int64",
    "bathRoom": "int64",
    "elevator": "category",
    "fiveYearsProperty": "category",
    "subway": "category",
    "district": "category"
}
# Add total_price as new column by multiplying price * square (we didn't have it in the original data, but was mentioned in the problem definition)
df_transactions['totalPrice'] = df_transactions.apply(lambda row: row.price * row.square, axis=1)

df_transactions.dropna(subset=['constructionTime', 'communityAverage'] ,inplace=True)
# Fill the DOM column with 0's
df_transactions['DOM']= df_transactions['DOM'].fillna(0)

# set the id as index
try:
    df_transactions.set_index('id', inplace=True)
except KeyError:
    print("id already set as index")
# Fix the date column to datetime
df_transactions['tradeTime'] = pd.to_datetime(df_transactions['tradeTime'], format='%Y-%m-%d')
# turn date time into int (the epoch)
df_transactions['tradeTime'] = [timestamp.value for timestamp in df_transactions['tradeTime']]
# Use above type columns to convert data types
df_transactions = df_transactions.astype(types)
# Now let's encode all the categorical data back to ints (little redundant, but it's a good check)
for col in df_transactions.select_dtypes(include='category'):
    encoder = LabelEncoder()
    encoder.fit(df_transactions[col])
    encoder_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
    df_transactions[col] = encoder.fit_transform(df_transactions[col])
    # Check we got the right amount of unique values and print the encoder to see mapping
    print(f"{col} with values: {np.sort(df_transactions[col].unique())}")
    print(encoder_mapping)

# double check everything is good
df_transactions.info()


/tmp/ipykernel_9620/3411723646.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions['totalPrice'] = df_transactions.apply(lambda row: row.price * row.square, axis=1)
/tmp/ipykernel_9620/3411723646.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions.dropna(subset=['constructionTime', 'communityAverage'] ,inplace=True)
/tmp/ipykernel_9620/3411723646.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

Cid with values: [   0    1    2 ... 3768 3769 3770]
{1111027373683.0: 0, 1111027373684.0: 1, 1111027373685.0: 2, 1111027373686.0: 3, 1111027373689.0: 4, 1111027373690.0: 5, 1111027373691.0: 6, 1111027373694.0: 7, 1111027373695.0: 8, 1111027373696.0: 9, 1111027373697.0: 10, 1111027373698.0: 11, 1111027373705.0: 12, 1111027373708.0: 13, 1111027373709.0: 14, 1111027373716.0: 15, 1111027373720.0: 16, 1111027373722.0: 17, 1111027373728.0: 18, 1111027373731.0: 19, 1111027373732.0: 20, 1111027373733.0: 21, 1111027373735.0: 22, 1111027373737.0: 23, 1111027373738.0: 24, 1111027373740.0: 25, 1111027373742.0: 26, 1111027373750.0: 27, 1111027373753.0: 28, 1111027373756.0: 29, 1111027373759.0: 30, 1111027373761.0: 31, 1111027373774.0: 32, 1111027373783.0: 33, 1111027373786.0: 34, 1111027373788.0: 35, 1111027373797.0: 36, 1111027373798.0: 37, 1111027373799.0: 38, 1111027373802.0: 39, 1111027373806.0: 40, 1111027373808.0: 41, 1111027373809.0: 42, 1111027373812.0: 43, 1111027373813.0: 44, 11110273738

## Encoding mapping
It's mapping pretty straightforward for all the numbers, it's all shifted down if not started with 0. Only mapping to potentially keep in mind is: `{'bottom': 0, 'high': 1, 'low': 2, 'middle': 3, 'top': 4, 'nan': 5}`.
One-hot encoding could be better (depending on approach used) for this column, we'll do that in the next notebook.

### Save data and move to new file for visualizations and pre-processing

In [27]:
df_transactions.to_csv(str(Path('data') / 'transactions_cleaned.csv'))